In [14]:
import numpy as np
import pandas as pd

###################################
# 1. DATA CONFIGURATION
###################################
# Load the data
file_path = "ScanRecords.csv"
df = pd.read_csv(file_path)

# Filter Type 1 and Type 2 patient data
type1_data = df[df["PatientType"] == "Type 1"]
type2_data = df[df["PatientType"] == "Type 2"]

# Extract durations and times for resampling
type1_durations = type1_data["Duration"].values
call_times_type1 = type1_data["Time"].values
type2_durations = type2_data["Duration"].values
call_times_type2 = type2_data["Time"].values

# Statistical properties for simulation
T1_MU_CI = (0.423, 0.4425)  # 95% CI for Type 1 mean duration
T1_SIGMA_CI = (0.0911, 0.1046)  # 95% CI for Type 1 std deviation
T2_MU_CI = (0.6464, 0.6943)  # 95% CI for Type 2 mean duration
T2_SIGMA = 0.15  # Fixed std deviation for Type 2

# Confidence intervals for patient arrivals
T1_ARRIVAL_CI = (14.87, 18.05)  # Type 1 daily arrivals
T2_ARRIVAL_CI = (9.925, 10.920)  # Type 2 daily arrivals

###################################
# 2. SIMULATION CONFIGURATION
###################################
NUM_DAYS = 20
OPEN_TIME = 8.0
CLOSE_TIME = 17.0
MINUTES_PER_DAY = (CLOSE_TIME - OPEN_TIME) * 60  # 540 minutes
TIMESLOT_T1 = 30  # Fixed timeslot length for Type 1 (minutes)
TIMESLOT_T2 = 40  # Fixed timeslot length for Type 2 (minutes)

###################################
# 3. HELPER FUNCTIONS
###################################
def generate_type1_day():
    """
    Generate Type 1 patient data for a day: call times and durations.
    """
    num_arrivals = np.random.poisson(np.random.uniform(*T1_ARRIVAL_CI))
    call_times = np.random.uniform(0, MINUTES_PER_DAY, size=num_arrivals)
    call_times.sort()
    mu_t1 = np.random.uniform(*T1_MU_CI)
    sigma_t1 = np.random.uniform(*T1_SIGMA_CI)
    durations = np.random.normal(mu_t1, sigma_t1, num_arrivals)
    durations = np.clip(durations, 0.2406, 0.6249) * 60  # Prediction interval in minutes
    return list(zip(call_times, durations))

def generate_type2_day():
    """
    Generate Type 2 patient data for a day: call times and durations.
    """
    num_arrivals = np.random.choice(np.arange(9, 13))  # Type 2 arrivals based on CI
    call_times = np.random.uniform(0, MINUTES_PER_DAY, size=num_arrivals)
    call_times.sort()
    mu_t2 = np.random.uniform(*T2_MU_CI)
    durations = np.random.normal(mu_t2, T2_SIGMA, num_arrivals)
    durations = np.clip(durations, 0.221, 1.147) * 60  # Realistic range in minutes
    return list(zip(call_times, durations))

def schedule_patients(arrivals, timeslot_length, start_day=1):
    """
    Schedule patients starting from the next working day.
    """
    schedule = []
    current_time = 0.0
    for call_time, duration in arrivals:
        start_time = max(current_time, timeslot_length)
        end_time = start_time + timeslot_length
        schedule.append((start_time, end_time, duration, start_day))
        current_time = end_time
    return schedule

###################################
# 4. MAIN SIMULATION
###################################
# Simulate for NUM_DAYS
type1_schedule = []
type2_schedule = []
for day in range(NUM_DAYS):
    t1_day = generate_type1_day()
    t2_day = generate_type2_day()

    type1_schedule.extend(schedule_patients(t1_day, TIMESLOT_T1, start_day=day + 1))
    type2_schedule.extend(schedule_patients(t2_day, TIMESLOT_T2, start_day=day + 1))

# Results
print("Type 1 Schedule:", type1_schedule)
print("Type 2 Schedule:", type2_schedule)


Type 1 Schedule: [(30, 60, 19.541782399616466, 1), (60, 90, 19.988693872044973, 1), (90, 120, 27.864921407039574, 1), (120, 150, 28.627289234117516, 1), (150, 180, 24.709408292159686, 1), (180, 210, 26.616090919600254, 1), (210, 240, 27.219337383498324, 1), (240, 270, 31.247793655296444, 1), (270, 300, 23.50153646243576, 1), (300, 330, 36.83946564919932, 1), (330, 360, 26.506798065651374, 1), (360, 390, 24.2696651049379, 1), (390, 420, 20.95963751772173, 1), (420, 450, 29.814778194677064, 1), (450, 480, 22.293627976666784, 1), (480, 510, 31.529025174002594, 1), (510, 540, 21.892521025844832, 1), (540, 570, 20.851906680354016, 1), (570, 600, 16.536230805444355, 1), (600, 630, 33.42789412644205, 1), (30, 60, 24.15853827017058, 2), (60, 90, 21.149681022201158, 2), (90, 120, 31.599877039710506, 2), (120, 150, 28.316164338929642, 2), (150, 180, 33.51731582572817, 2), (180, 210, 22.084747916161774, 2), (210, 240, 23.881428811067714, 2), (240, 270, 28.166093931545188, 2), (270, 300, 37.494, 2

In [4]:
import numpy as np
import pandas as pd

###################################
# 1. DATA CONFIGURATION
###################################
# Statistical properties for simulation
T1_MU_CI = (0.423, 0.4425)  # 95% CI for Type 1 mean duration
T1_SIGMA_CI = (0.0911, 0.1046)  # 95% CI for Type 1 std deviation
T2_MU_CI = (0.6464, 0.6943)  # 95% CI for Type 2 mean duration
T2_SIGMA = 0.15  # Fixed std deviation for Type 2

# Confidence intervals for patient arrivals
T1_ARRIVAL_CI = (14.87, 18.05)  # Type 1 daily arrivals
T2_ARRIVAL_CI = (9.925, 10.920)  # Type 2 daily arrivals

# Simulation configuration
NUM_DAYS = 20
OPEN_TIME = 8.0
CLOSE_TIME = 17.0
MINUTES_PER_DAY = (CLOSE_TIME - OPEN_TIME) * 60  # 540 minutes
FIXED_TIMESLOT_T1 = 30  # Fixed timeslot length for Type 1 (minutes)
FIXED_TIMESLOT_T2 = 40  # Fixed timeslot length for Type 2 (minutes)

###################################
# 2. HELPER FUNCTIONS
###################################
def generate_type1_day():
    """
    Generate Type 1 patient data for a day: call times and durations.
    """
    num_arrivals = np.random.poisson(np.random.uniform(*T1_ARRIVAL_CI))
    call_times = np.random.uniform(0, MINUTES_PER_DAY, size=num_arrivals)
    call_times.sort()
    mu_t1 = np.random.uniform(*T1_MU_CI)
    sigma_t1 = np.random.uniform(*T1_SIGMA_CI)
    durations = np.random.normal(mu_t1, sigma_t1, num_arrivals)
    durations = np.clip(durations, 0.2406, 0.6249) * 60  # Prediction interval in minutes
    return list(zip(call_times, durations))

def generate_type2_day():
    """
    Generate Type 2 patient data for a day: call times and durations.
    """
    num_arrivals = np.random.choice(np.arange(9, 13))  # Type 2 arrivals based on CI
    call_times = np.random.uniform(0, MINUTES_PER_DAY, size=num_arrivals)
    call_times.sort()
    mu_t2 = np.random.uniform(*T2_MU_CI)
    durations = np.random.normal(mu_t2, T2_SIGMA, num_arrivals)
    durations = np.clip(durations, 0.221, 1.147) * 60  # Realistic range in minutes
    return list(zip(call_times, durations))

def schedule_old_system(arrivals, timeslot_length):
    """
    Schedule patients using fixed timeslots for the old system.
    """
    schedule = []
    current_time = 0.0
    for _, duration in arrivals:
        start_time = current_time
        end_time = start_time + timeslot_length
        schedule.append((start_time, end_time, duration))
        current_time = end_time
    return schedule

def schedule_new_system_with_buffer(type1_arrivals, type2_arrivals, buffer=5):
    """
    New system => 2 identical MRI machines (T1 adaptive slot, T2 adaptive slot).
    Assign to whichever machine is free first, with a buffer between slots.
    """
    combined = [(t, d, 'T1') for (t, d, _) in type1_arrivals] + \
               [(t, d, 'T2') for (t, d, _) in type2_arrivals]
    combined.sort(key=lambda x: x[0])

    sched_A = []
    sched_B = []
    current_A = 0.0
    current_B = 0.0

    for (call_time, duration, ptype) in combined:
        slot_len = duration + buffer  # Add buffer time
        if current_A <= current_B:
            start = max(current_A, call_time)
            end = start + slot_len
            sched_A.append((start, end, duration, ptype))
            current_A = end
        else:
            start = max(current_B, call_time)
            end = start + slot_len
            sched_B.append((start, end, duration, ptype))
            current_B = end
    
    return sched_A, sched_B

def calculate_metrics_with_noise(schedule, noise_range=(1, 5)):
    """
    Calculate finishing time, busy time, and overtime, incorporating random delays.
    """
    finishing_time = 0.0
    busy_time = 0.0
    for start, end, duration, *rest in schedule:
        random_delay = np.random.uniform(*noise_range)  # Simulate random delay
        finishing_time = max(finishing_time, end + random_delay)
        busy_time += duration
    overtime = max(0, finishing_time - MINUTES_PER_DAY)
    return finishing_time, busy_time, overtime


###################################
# 3. MAIN SIMULATION
###################################
results_old = []
results_new = []

for day in range(NUM_DAYS):
    # Generate daily arrivals
    t1_arrivals = generate_type1_day()
    t2_arrivals = generate_type2_day()

    # Old system scheduling
    sched_t1_old = schedule_old_system(t1_arrivals, FIXED_TIMESLOT_T1)
    sched_t2_old = schedule_old_system(t2_arrivals, FIXED_TIMESLOT_T2)

    finish_t1_old, busy_t1_old, ot_t1_old = calculate_metrics(sched_t1_old)
    finish_t2_old, busy_t2_old, ot_t2_old = calculate_metrics(sched_t2_old)

    finish_old = max(finish_t1_old, finish_t2_old)
    total_overtime_old = ot_t1_old + ot_t2_old
    results_old.append((finish_old, total_overtime_old))

    # New system scheduling
    sched_A_new, sched_B_new = schedule_new_system(t1_arrivals, t2_arrivals)

    finish_A_new, busy_A_new, ot_A_new = calculate_metrics(sched_A_new)
    finish_B_new, busy_B_new, ot_B_new = calculate_metrics(sched_B_new)

    finish_new = max(finish_A_new, finish_B_new)
    total_overtime_new = ot_A_new + ot_B_new
    results_new.append((finish_new, total_overtime_new))

# Summarize results
avg_finish_old = np.mean([x[0] for x in results_old])
avg_ot_old = np.mean([x[1] for x in results_old])
avg_finish_new = np.mean([x[0] for x in results_new])
avg_ot_new = np.mean([x[1] for x in results_new])

###################################
# 4. OUTPUT RESULTS
###################################
print("\n====== OLD SYSTEM RESULTS ======")
print(f"Avg finishing time (min from 8:00): {avg_finish_old:.2f}")
print(f"Avg daily overtime (minutes):      {avg_ot_old:.2f}")

print("\n====== NEW SYSTEM RESULTS ======")
print(f"Avg finishing time (min from 8:00): {avg_finish_new:.2f}")
print(f"Avg daily overtime (minutes):      {avg_ot_new:.2f}")



====== OLD SYSTEM RESULTS ======
Avg finishing time (min from 8:00): 554.50
Avg daily overtime (minutes):      43.50

====== NEW SYSTEM RESULTS ======
Avg finishing time (min from 8:00): 458.64
Avg daily overtime (minutes):      5.36
